<a href="https://colab.research.google.com/github/Mahdi-Saadati/Advanced-Machine-Learning-and-Deep-Neural-Networks/blob/main/9.a.%20Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl" align="right">

شبکه ترنسفورمر (Transformer)

1️⃣ ایده اصلی

فرض کنید می‌خواهیم یک متن را پردازش کنیم، مثلاً جمله:
"من امروز به مدرسه رفتم"

هدف ما این است که مدل بتواند بفهمد ارتباط بین کلمات چیست، حتی اگر کلمات فاصله زیادی داشته باشند.

قبل از Transformer، مدل‌های قدیمی مثل RNN یا LSTM کلمات را یکی یکی پردازش می‌کردند و اگر یک کلمه خیلی دور بود، یادگیری ارتباط سخت می‌شد.

Transformer می‌گوید:

"چرا همه چیز را یکجا نبینیم و مستقیماً ارتباط بین هر کلمه با هر کلمه را بسنجیم؟"

2️⃣ بخش‌های کلیدی Transformer
🔹 ورودی: کلمات → بردارها

هر کلمه تبدیل می‌شود به یک عدد یا بردار عددی (مثل شناسه کلمه یا embedding).
مثال: "من" → [0.1, 0.3, 0.5]
"مدرسه" → [0.7, 0.2, 0.9]

🔹 Attention (توجه)

اینجاست که جادو اتفاق می‌افتد:

به هر کلمه می‌گوید: "به کدام کلمات دیگر جمله توجه کنم تا معنی‌ام را بهتر بفهمم؟"

مثال:

کلمه "رفتم" بیشتر به "مدرسه" توجه می‌کند تا "امروز"، چون معنی رفتن به مقصد مرتبط است.

این کار با محاسبه وزن‌ها بین همه کلمات انجام می‌شود.

وزن زیاد = ارتباط قوی

وزن کم = ارتباط ضعیف

این بخش Self-Attention نام دارد.

🔹 Feed-Forward (لایه پردازشی)

بعد از Attention، هر کلمه با اطلاعاتی که از دیگر کلمات گرفته ترکیب می‌شود و از چند لایه ساده پردازش عبور می‌کند تا اطلاعات بهتر یاد گرفته شود.

🔹 Positional Encoding

چون Transformer همه کلمات را همزمان پردازش می‌کند، باید بداند ترتیب کلمات چیست.

این کار با اضافه کردن یک بردار موقعیت به هر کلمه انجام می‌شود.
مثال:
"من" → [0.1,0.3,0.5] + position_0
"مدرسه" → [0.7,0.2,0.9] + position_3

🔹 Encoder و Decoder

Encoder: متن ورودی را می‌فهمد و اطلاعات مهم را استخراج می‌کند.

Decoder: بر اساس اطلاعات Encoder، متن خروجی می‌سازد (مثل ترجمه یا پاسخ).

3️⃣ خلاصه بصری ساده
ورودی متن → تبدیل به بردار → Attention (هر کلمه به همه نگاه می‌کند)
→ لایه‌های پردازش → خروجی معنایی (یا ترجمه، یا پیش‌بینی کلمه بعدی)


بزرگترین قدرت Transformer این است که می‌تواند ارتباطات دور بین کلمات را به راحتی یاد بگیرد، بدون نیاز به پردازش کلمه به کلمه.

💡 مثال روزمره:
فرض کن می‌خواهی بفهمی جمله‌ی "علی که دیروز مریض بود امروز به مدرسه رفت" چه معنایی دارد.

Transformer می‌تواند بفهمد "رفتن" مربوط به "علی" و "مدرسه" است، حتی اگر کلمه‌ها فاصله زیادی داشته باشند.

In [14]:
from keras.layers import TextVectorization, Input, Embedding, MultiHeadAttention, LayerNormalization, Dense
from keras.models import Model
import numpy as np

# 1. نمونه جملات انگلیسی و ترجمه فارسی
eng_texts = [
    "I love AI",
    "Deep learning is fun",
    "Transformers are powerful",
]
fa_texts = [
    "من هوش مصنوعی را دوست دارم",
    "یادگیری عمیق جذاب است",
    "ترنسفورمرها قدرتمند هستند",
]

# 2. TextVectorization برای انگلیسی و فارسی
max_len = 6
eng_vectorizer = TextVectorization(max_tokens=50, output_sequence_length=max_len)
eng_vectorizer.adapt(eng_texts)
fa_vectorizer = TextVectorization(max_tokens=50, output_sequence_length=max_len)
fa_vectorizer.adapt(fa_texts)

eng_vocab = eng_vectorizer.get_vocabulary()
fa_vocab = fa_vectorizer.get_vocabulary()

# 3. آماده‌سازی داده‌ها
eng_seq = eng_vectorizer(eng_texts).numpy()[:, :-1]
fa_seq = fa_vectorizer(fa_texts).numpy()[:, 1:]  # خروجی (shifted)

# 4. تعریف مدل ساده Transformer
embed_dim = 16
num_heads = 2
ff_dim = 32

# Encoder
encoder_inputs = Input(shape=(max_len-1,))
x = Embedding(input_dim=len(eng_vocab), output_dim=embed_dim)(encoder_inputs)
attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
x = LayerNormalization(epsilon=1e-6)(x + attn_output)
ff = Dense(ff_dim, activation="relu")(x)
ff = Dense(embed_dim)(ff)
encoder_outputs = LayerNormalization(epsilon=1e-6)(x + ff)

# Decoder
decoder_inputs = Input(shape=(max_len-1,))
y = Embedding(input_dim=len(fa_vocab), output_dim=embed_dim)(decoder_inputs)
attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(y, encoder_outputs)
y = LayerNormalization(epsilon=1e-6)(y + attn_output)
ff = Dense(ff_dim, activation="relu")(y)
ff = Dense(embed_dim)(ff)
decoder_outputs = LayerNormalization(epsilon=1e-6)(y + ff)

# Output layer
outputs = Dense(len(fa_vocab), activation="softmax")(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

# 5. آموزش مدل (برای مثال کوچک فقط یک نمونه)
model.fit([eng_seq, fa_seq], np.expand_dims(fa_seq, -1), epochs=500, verbose=0)

# 6. پیش‌بینی ترجمه
test_sentence = "I love AI"
test_seq = eng_vectorizer([test_sentence]).numpy()[:, :-1]
decoder_input = np.zeros((1, max_len-1))  # شروع با صفرها
pred = model.predict([test_seq, decoder_input])

pred_indices = np.argmax(pred[0], axis=-1)
pred_words = [fa_vocab[i] for i in pred_indices if i != 0]
translation = " ".join(pred_words)
print("English:", test_sentence)
print("Predicted Persian:", translation)


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, 5, 16)     │        192 │ input_layer_11[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 5, 16)     │      2,160 │ embedding_11[0][… │
│ (MultiHeadAttentio… │                   │            │ embedding_11[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_22 (Add)        │ (None, 5, 16)     │          0 │ embedding_11[0][… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 16)     │         32 │ add_22[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_31 (Dense)    │ (None, 5, 32)     │        544 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_32 (Dense)    │ (None, 5, 16)     │        528 │ dense_31[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_12      │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_23 (Add)        │ (None, 5, 16)     │          0 │ layer_normalizat… │
│                     │                   │            │ dense_32[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_12        │ (None, 5, 16)     │        240 │ input_layer_12[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 16)     │         32 │ add_23[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 5, 16)     │      2,160 │ embedding_12[0][… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_24 (Add)        │ (None, 5, 16)     │          0 │ embedding_12[0][… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 16)     │         32 │ add_24[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 5, 32)     │        544 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_34 (Dense)    │ (None, 5, 16)     │        528 │ dense_33[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_25 (Add)        │ (None, 5, 16)     │          0 │ layer_normalizat… │
│                     │                   │            │ dense_34[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 16)     │         32 │ add_25[0][0]      │
│ (LayerNormalizatio… │                   │            │                 

 Total params: 7,279 (28.43 KB)

 Trainable params: 7,279 (28.43 KB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 909ms/step
English: I love AI
Predicted Persian: 
